PassengerId & Ticket - don't add value to the prediction since they are unique identification values

Pclass - important ordinal attribute

Fare - important ordinal attribute

Name - we could infer new attributes from 'Mr', 'Mrs', 'Miss', 'Master' (see other values)

Sex - nominal

Age - quantitative

SibSp - # of siblings/spouse

Parch - # of parents/childrens

Cabin - cabin type: to exclude since ~80% empty

Embarked - port of embarcation

Survived - to predict

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def summary_missing_data(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['# of Missing', 'Percent'])
    return missing_data

In [3]:
raw = pd.read_csv('./data/train.csv')
rawt = pd.read_csv('./data/test.csv')

In [4]:
raw.shape, rawt.shape

((891, 12), (418, 11))

In [5]:
891+418

1309

In [6]:
brk = raw.shape[0]
rawfull = pd.concat([raw, rawt], ignore_index=True)
rawfull.shape

(1309, 12)

In [7]:
rawfull.iloc[brk-2:brk+2, :]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
889,26.0,C148,C,30.0000,"Behr, Mr. Karl Howell",0,890,1,male,0,1.0,111369
890,32.0,NaN,Q,7.7500,"Dooley, Mr. Patrick",0,891,3,male,0,0.0,370376
891,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911
892,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272


In [8]:
summary_missing_data(rawfull.drop(columns=['Survived'])).head(6)

,# of Missing,Percent
Cabin,1014,0.774637
Age,263,0.200917
Embarked,2,0.001528
Fare,1,0.000764
Ticket,0,0.000000
SibSp,0,0.000000


# Attributes that don't add value to our predictor
- **PassengerId** and **Ticket** don't add value to the prediction since they are unique identification values
- **Cabin** to exclude since ~80% empty

In [9]:
dataset = rawfull.drop(columns=['Cabin', 'PassengerId', 'Ticket'])

# Missing values

- How to deal with **Age**?
    - Fill mode? median by groups?
    
    
- Who are the ones with no **Embarked** and **Fare**?
    - Decide what to do
    - Fill the **Embarked** and **Fare** type of other similar persons (by age? parch? pclass?)

In [10]:
dataset[ pd.isna(dataset['Embarked']) ]

,Age,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived
61,38.0,NaN,80.0,"Icard, Miss. Amelie",0,1,female,0,1.0
829,62.0,NaN,80.0,"Stone, Mrs. George Nelson (Martha Evelyn)",0,1,female,0,1.0


In [11]:
rawfull[ pd.isna(rawfull['Fare']) ]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
1043,60.5,NaN,S,NaN,"Storey, Mr. Thomas",0,1044,3,male,0,NaN,3701


## Fill nulls with

In [12]:
fillers = {
    'Age': raw['Age'].median(),
    'Embarked': raw['Embarked'].mode()[0],
    'Fare': raw['Fare'].mode()[0]
}
dataset = dataset.fillna(fillers)
summary_missing_data( dataset ).head()

,# of Missing,Percent
Survived,418,0.319328
SibSp,0,0.000000
Sex,0,0.000000
Pclass,0,0.000000
Parch,0,0.000000


In [13]:
dataset.iloc[brk-2:brk+2, :].head()

,Age,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived
889,26.0,C,30.0000,"Behr, Mr. Karl Howell",0,1,male,0,1.0
890,32.0,Q,7.7500,"Dooley, Mr. Patrick",0,3,male,0,0.0
891,34.5,Q,7.8292,"Kelly, Mr. James",0,3,male,0,NaN
892,47.0,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,3,female,1,NaN


## Identify from the **Name** meaningful socio-economic status as 'Mr', 'Mrs', 'Dr', 'Master', etc appears. Do OneHot encoder on them, then delete column Name

In [14]:
titles = []
for n in dataset['Name']:
    s = n.split(',')
    status = s[1].split('.')
    titles.append(status[0].strip())
titles = list(set(titles))
titles

['Ms',
 'Sir',
 'Don',
 'Miss',
 'Col',
 'Mr',
 'Jonkheer',
 'Dr',
 'Mlle',
 'Mrs',
 'Rev',
 'Major',
 'Lady',
 'the Countess',
 'Capt',
 'Master',
 'Dona',
 'Mme']

In [15]:
titles_encoded = []
for a_title in titles:
    titles_encoded.append(pd.DataFrame([ 1 if n.find(a_title+'.') >= 0 else 0 for n in dataset['Name'] ], columns=[a_title]))

In [16]:
datasetdummy = pd.concat([dataset, *titles_encoded], axis=1)
datasetdummy = datasetdummy.drop(columns=['Name'])
datasetdummy.shape

(1309, 26)

In [17]:
datasetdummy.head(5)

,Age,Embarked,Fare,Parch,Pclass,Sex,SibSp,Survived,Ms,Sir,...,Mlle,Mrs,Rev,Major,Lady,the Countess,Capt,Master,Dona,Mme
0,22.0,S,7.2500,0,3,male,1,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,38.0,C,71.2833,0,1,female,1,1.0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,26.0,S,7.9250,0,3,female,0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,35.0,S,53.1000,0,1,female,1,1.0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,35.0,S,8.0500,0,3,male,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
datasetdummy = pd.get_dummies(datasetdummy, columns=['Embarked', 'Parch', 'Pclass', 'Sex', 'SibSp'])


In [19]:
summary_missing_data(datasetdummy).head(6)

,# of Missing,Percent
Survived,418,0.319328
SibSp_8,0,0.000000
Mlle,0,0.000000
Dona,0,0.000000
Master,0,0.000000
Capt,0,0.000000


In [20]:
datasetdummy.iloc[brk-2:brk+2, :].head()

,Age,Fare,Survived,Ms,Sir,Don,Miss,Col,Mr,Jonkheer,...,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
889,26.0,30.0000,1.0,0,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
890,32.0,7.7500,0.0,0,0,0,0,0,1,0,...,1,0,1,1,0,0,0,0,0,0
891,34.5,7.8292,NaN,0,0,0,0,0,1,0,...,1,0,1,1,0,0,0,0,0,0
892,47.0,7.0000,NaN,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0


In [21]:
print('Train shape \t\t{}'.format(raw.shape))
print('Test shape \t\t{}'.format(rawt.shape))
# print('Combined Raw shape \t{}'.format(rawfull.shape))
print('Dummy shape \t\t{}'.format(datasetdummy.shape))

Train shape 		(891, 12)
Test shape 		(418, 11)
Dummy shape 		(1309, 44)


## Split training and test set and save

In [22]:
datatrain = datasetdummy.iloc[0:brk,:]
datatrain.shape

(891, 44)

In [23]:
datatest = datasetdummy.iloc[brk:,:].drop(columns=['Survived'])
datatest.shape

(418, 43)

In [28]:
raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
rawt.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [26]:
datatest.head()

,Age,Fare,Ms,Sir,Don,Miss,Col,Mr,Jonkheer,Dr,...,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
891,34.5,7.8292,0,0,0,0,0,1,0,0,...,1,0,1,1,0,0,0,0,0,0
892,47.0,7.0000,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
893,62.0,9.6875,0,0,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
894,27.0,8.6625,0,0,0,0,0,1,0,0,...,1,0,1,1,0,0,0,0,0,0
895,22.0,12.2875,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0


In [25]:
datatrain.to_csv('cl_train.csv', index=False)
datatest.to_csv('cl_test.csv', index=False)